__Generators__: a type of iterator

__Generator functions__: generator factories; return a generator

__Generator expressions__: more concise way of creating generators, uses comprehension syntax

### Yielding and Generator Functions

__Yield__

The `yield` keyword:
- emits a value
- effectively suspends the function (but retains current state)
- calling `next` resumes the function after the `yield`
- will raise StopIteration if `next` is called after function returns

__Generators__

A function that contains the `yield` statement, is called a *generator function*. These are normal functions, but calling them returns a generator object. Thus, we can think of these functions as generator factories.

The function body will execute until it encounters a `yield` statement, it yields the value (as the return of `next`) then suspends itself until `next` is called again.

In [5]:
def my_gen():
    yield 'one'
    yield 'two'
    yield 'three'

In [6]:
# The generator object is created when we call the function
gen = my_gen() #gen is a generator

In [7]:
# And is executed when we call next() on the generator
next(gen)
next(gen)
next(gen)

'three'

In [8]:
next(gen)

StopIteration: 

Generators are *iterators*, since they implement the iterator protocol by having `__iter__` and `__next__` methods.

In [9]:
iter(gen) is gen

True

In [10]:
import math

# Remember the Factorial Iterator
class FactIter:
    def __init__(self, n):
        self.n = n
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i += 1
            return result
        
fact_iter = FactIter(5)

In [11]:
# We can rewrite this as a generator
def factorials(n):
    for i in range(n):
        yield math.factorial(i)
        
fact_iter = factorials(5)

### Iterables from Generators

Since generators are iterators, to create an iterable all we have to do is create the class (that implements iterable) which will return new iterators every time.

In [12]:
# The generator
def squares(n):
    for i in range(n):
        yield i ** 2

In [15]:
class Squares:
    def __init__(self, n):
        self.n = n
        
    def __iter__(self):
        return squares(self.n)

In [17]:
sq = Squares(5)

list(sq)

[0, 1, 4, 9, 16]

In [19]:
list(sq) # sq will not be exhausted because a new iterator was created

[0, 1, 4, 9, 16]

### Generator Expressions

Generator expresssion use the same syntax as comprehensions (except using () instead of []), and have all the same functionality, such as nesting and conditionals.

| `[i ** 2 for i in range(5)]` | `(i ** 2 for i in range(5))`
| ---------------------------- | ----------------------------
| a list is returned           | a generator is returned
| evaluation is eager          | evaluation is lazy
| has local scope              | has local scope
| can access nonlocal/global   | can access nonlocal/global
| iterable                     | iterator

__Resource Utilization__

In general, list comprehensions:
- load entire collection in memory
- take longer to create
- but iteration is faster

While generator expressions:
- only load a single object in memory at a time
- create objects when requested
- but iteration is slower

If iterating through all the elements, performance is about the same. But if only iterating partially through a collection, generator expressions are more efficient. In general, generators tend to have less memory overhead.

### Yield From

Often we may need to delegate yielding elements to another iterator.

```
def read_all_data():
    for file in ('file1.csv', 'file2.csv'):
        with open(file) as f:
            for line in f:
                yield line
```

The inner loop is basically using the file iterator and yielding values directly. We are essentially delegating yielding to the file iterator.

We can replace the inner loop using `yield from`:

```
def read_all_data():
    for file in ('file1.csv', 'file2.csv'):
        with open(file) as f:
            yield from f
```